# Delays prediction - machine learning models (regression)

Katarzyna Mocio 429956 
Marcin Miszkiel 432418

# 1. Prepare necessary libraries


In [28]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
pd.set_option("display.max_columns",100)

# Data wrangling
import numpy as np
from datetime import datetime as dt 

# Visualizations
import seaborn as sns
import matplotlib.pyplot as plt 
%matplotlib inline

# Modelling with scikit-learn
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error, mean_absolute_error, r2_score, make_scorer
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import ElasticNetCV
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import TweedieRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.inspection import permutation_importance
import optuna
import sklearn



Delays for regression
Your task is to apply various ML algorithms (see the rules below) to build a model explaining the delays of the flights based on the training sample and generate predictions for all observations from the test sample.

The dataset includes 1631327 observations in the training sample and 407832 in the test sample and the following columns:

Weekday – Day of the week when the flight occurred (1 for Sunday, 7 for Saturday).
Month_of_Year – The numerical month (1-12) when the flight took place.
Day_of_Month – The day of the month (1-31) when the flight occurred.
Scheduled_Departure_Time – The scheduled local time of flight departure.
Scheduled_Arrival_Time – The scheduled local time of flight arrival.
Marketing_Airline – The airline code under which the flight was marketed.
Marketing_Airline_DOT_ID – Department of Transportation identifier for the marketing airline.
Flight_Number – The number assigned to the flight by the operating airline.
Origin_Airport_ID – Unique identifier for the departure airport.
Destination_Airport_ID – Unique identifier for the arrival airport.
Flight_Cancelled – Indicator of whether the flight was cancelled (1 = Yes, 0 = No).
Departure_State – The state code of the departure location.
Arrival_State – The state code of the arrival location.
Departure_Delay – Total delay in minutes at departure.
Diverted_Airport_Landings – Count of unplanned landings at other airports.
Taxi_Out_Time – Time in minutes from gate departure until takeoff.
Taxi_In_Time – Time in minutes from landing to gate arrival.
Flight_Diverted – Indicator of whether the flight was diverted (1 = Yes, 0 = No).
Actual_Departure_Time – The actual local time the flight departed.
Flight_Duration – The duration of the flight in minutes from takeoff to landing.
Flight_Distance – The total distance covered by the flight in miles.
Origin_Temperature – The temperature at the origin airport at the time of the flight’s departure.
Destination_Temperature – The temperature at the destination airport at the time of the flight’s arrival.
Origin_Wind_Speed – The wind speed at the origin airport during the departure of the flight.
Destination_Wind_Speed – The wind speed at the destination airport during the departure of the flight.
Origin_Precipitation – The amount of precipitation, such as rain or snow, at the origin airport around the flight’s departure time.
Destination_Precipitation – The amount of precipitation, such as rain or snow, at the destination airport around the flight’s arrival time.
Arrival_Delay – Total delay in minutes at arrival (outcome variable, only in the training sample)

Let's load the data and look at the first five observations

In [58]:
delays = pd.read_csv('delays_train.csv')

delays.head()

,Weekday,Month_of_Year,Day_of_Month,Scheduled_Departure_Time,Scheduled_Arrival_Time,Marketing_Airline,Marketing_Airline_DOT_ID,Flight_Number,Origin_Airport_ID,Destination_Airport_ID,Flight_Cancelled,Departure_State,Arrival_State,Departure_Delay,Arrival_Delay,Diverted_Airport_Landings,Taxi_Out_Time,Taxi_In_Time,Flight_Diverted,Actual_Departure_Time,Flight_Duration,Flight_Distance,Origin_Temperature,Destination_Temperature,Origin_Wind_Speed,Destination_Wind_Speed,Origin_Precipitation,Destination_Precipitation
0,6,6,25,1222,1444,B6,20409,520,10397,12478.0,False,GA,NY,NaN,NaN,0,16.0,10.0,False,1224.0,107.0,760.0,25.140220,19.357739,21.019808,15.452723,0.0,0.0
1,2,6,21,1216,1304,NaN,19805,6297,14107,15376.0,False,AZ,AZ,0.088687,-4.178483,0,16.0,5.0,False,1216.0,23.0,110.0,13.279939,20.470690,18.045064,12.910265,0.0,0.0
2,3,1,5,1945,2055,DL,19790,4124,13487,13076.0,False,MN,WI,NaN,5.042185,0,34.0,5.0,False,1945.0,36.0,119.0,17.816202,13.967273,21.606228,17.976362,0.0,0.1
3,2,3,22,700,924,AA,19805,1538,15624,11298.0,False,FL,TX,-1.802698,-0.206932,0,10.0,11.0,False,658.0,125.0,641.0,24.562566,14.509228,24.946489,22.630553,0.0,0.0
4,4,7,14,2130,2359,NaN,19930,1116,14747,12889.0,False,WA,NV,NaN,14.006092,0,23.0,7.0,False,2133.0,130.0,867.0,8.817992,10.866812,17.426336,17.401007,0.0,0.0


In [59]:
delays.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1631327 entries, 0 to 1631326
Data columns (total 28 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   Weekday                    1631327 non-null  int64  
 1   Month_of_Year              1631327 non-null  int64  
 2   Day_of_Month               1631327 non-null  int64  
 3   Scheduled_Departure_Time   1631327 non-null  int64  
 4   Scheduled_Arrival_Time     1631327 non-null  int64  
 5   Marketing_Airline          1467776 non-null  object 
 6   Marketing_Airline_DOT_ID   1631327 non-null  int64  
 7   Flight_Number              1631327 non-null  int64  
 8   Origin_Airport_ID          1631327 non-null  int64  
 9   Destination_Airport_ID     1468233 non-null  float64
 10  Flight_Cancelled           1631327 non-null  bool   
 11  Departure_State            1631327 non-null  object 
 12  Arrival_State              1631327 non-null  object 
 13  Departure_De

In [60]:
an = pd.read_csv('airline_name.csv', index_col=False)
an = an.set_index('id')['name'].to_dict()
an

{19805: 'AA',
 19930: 'AS',
 20409: 'B6',
 19790: 'DL',
 20436: 'F9',
 20368: 'G4',
 19690: 'HA',
 20416: 'NK',
 19977: 'UA',
 19393: 'WN'}

In [61]:
delays['Marketing_Airline'] = delays['Marketing_Airline_DOT_ID'].map(an)


In [62]:
delays.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1631327 entries, 0 to 1631326
Data columns (total 28 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   Weekday                    1631327 non-null  int64  
 1   Month_of_Year              1631327 non-null  int64  
 2   Day_of_Month               1631327 non-null  int64  
 3   Scheduled_Departure_Time   1631327 non-null  int64  
 4   Scheduled_Arrival_Time     1631327 non-null  int64  
 5   Marketing_Airline          1631327 non-null  object 
 6   Marketing_Airline_DOT_ID   1631327 non-null  int64  
 7   Flight_Number              1631327 non-null  int64  
 8   Origin_Airport_ID          1631327 non-null  int64  
 9   Destination_Airport_ID     1468233 non-null  float64
 10  Flight_Cancelled           1631327 non-null  bool   
 11  Departure_State            1631327 non-null  object 
 12  Arrival_State              1631327 non-null  object 
 13  Departure_De

In [64]:
grouped_df = delays.groupby(['Marketing_Airline', 'Flight_Number', 'Origin_Airport_ID', 'Destination_Airport_ID', 'Weekday', 'Scheduled_Departure_Time', 'Scheduled_Arrival_Time'])
value_counts = grouped_df['Marketing_Airline'].value_counts()
value_counts

Marketing_Airline  Flight_Number  Origin_Airport_ID  Destination_Airport_ID  Weekday  Scheduled_Departure_Time  Scheduled_Arrival_Time
AA                 1              12478              12892.0                 1        730                       1025                      2
                                                                                                                1041                      2
                                                                                                                1054                      2
                                                                                                                1057                      2
                                                                                                                1104                      1
                                                                                                                                         ..
WN                 6930  

In [65]:
df = value_counts.reset_index()
df.columns = ['Marketing_Airline', 'Flight_Number', 'Origin_Airport_ID', 'Destination_Airport_ID', 'Weekday', 'Scheduled_Departure_Time', 'Scheduled_Arrival_Time', 'Count']

# Zapis do pliku Excel
df.to_csv('flights_count.csv', index=False)


In [66]:
airportD = pd.read_csv('flights_count.csv', index_col=False)
airportD

,Marketing_Airline,Flight_Number,Origin_Airport_ID,Destination_Airport_ID,Weekday,Scheduled_Departure_Time,Scheduled_Arrival_Time,Count
0,AA,1,12478,12892.0,1,730,1025,2
1,AA,1,12478,12892.0,1,730,1041,2
2,AA,1,12478,12892.0,1,730,1054,2
3,AA,1,12478,12892.0,1,730,1057,2
4,AA,1,12478,12892.0,1,730,1104,1
...,...,...,...,...,...,...,...,...
764190,WN,6930,12889,13495.0,6,930,1450,2
764191,WN,6931,13495,12889.0,6,1525,1715,2
764192,WN,6948,10821,10529.0,7,2020,2130,1
764193,WN,6951,13204,10397.0,3,2050,2225,1


In [67]:
delays['key'] = delays['Marketing_Airline'] + '_' + delays['Flight_Number'].astype(str) + '_' + delays['Origin_Airport_ID'].astype(str) + '_' + delays['Weekday'].astype(str) + '_' + delays['Scheduled_Departure_Time'].astype(str) + '_' + delays['Scheduled_Arrival_Time'].astype(str)
airportD['key'] = airportD['Marketing_Airline'] + '_' + airportD['Flight_Number'].astype(str) + '_' + airportD['Origin_Airport_ID'].astype(str) + '_' + airportD['Weekday'].astype(str) + '_' + airportD['Scheduled_Departure_Time'].astype(str) + '_' + airportD['Scheduled_Arrival_Time'].astype(str)

In [14]:
airportD.to_excel('flights_count.xlsx', index=False)

In [68]:
# Stworzenie słownika mapowania
map_dict = airportD.set_index('key')['Destination_Airport_ID'].to_dict()

map_dict

{'AA_1_12478_1_730_1025': 12892.0,
 'AA_1_12478_1_730_1041': 12892.0,
 'AA_1_12478_1_730_1054': 12892.0,
 'AA_1_12478_1_730_1057': 12892.0,
 'AA_1_12478_1_730_1104': 12892.0,
 'AA_1_12478_2_730_1025': 12892.0,
 'AA_1_12478_2_730_1041': 12892.0,
 'AA_1_12478_2_730_1050': 12892.0,
 'AA_1_12478_2_730_1054': 12892.0,
 'AA_1_12478_2_730_1057': 12892.0,
 'AA_1_12478_2_730_1104': 12892.0,
 'AA_1_12478_3_730_1025': 12892.0,
 'AA_1_12478_3_730_1041': 12892.0,
 'AA_1_12478_3_730_1050': 12892.0,
 'AA_1_12478_3_730_1054': 12892.0,
 'AA_1_12478_3_730_1104': 12892.0,
 'AA_1_12478_4_730_1025': 12892.0,
 'AA_1_12478_4_730_1041': 12892.0,
 'AA_1_12478_4_730_1050': 12892.0,
 'AA_1_12478_4_730_1054': 12892.0,
 'AA_1_12478_4_730_1057': 12892.0,
 'AA_1_12478_4_730_1104': 12892.0,
 'AA_1_12478_5_730_1025': 12892.0,
 'AA_1_12478_5_730_1041': 12892.0,
 'AA_1_12478_5_730_1050': 12892.0,
 'AA_1_12478_5_730_1054': 12892.0,
 'AA_1_12478_5_730_1057': 12892.0,
 'AA_1_12478_5_730_1104': 12892.0,
 'AA_1_12478_6_730_1

In [69]:
# Uzupełnianie brakujących wartości w 'delays'
delays['Destination_Airport_ID'] = delays['Destination_Airport_ID'].fillna(delays['key'].map(map_dict))
delays.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1631327 entries, 0 to 1631326
Data columns (total 29 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   Weekday                    1631327 non-null  int64  
 1   Month_of_Year              1631327 non-null  int64  
 2   Day_of_Month               1631327 non-null  int64  
 3   Scheduled_Departure_Time   1631327 non-null  int64  
 4   Scheduled_Arrival_Time     1631327 non-null  int64  
 5   Marketing_Airline          1631327 non-null  object 
 6   Marketing_Airline_DOT_ID   1631327 non-null  int64  
 7   Flight_Number              1631327 non-null  int64  
 8   Origin_Airport_ID          1631327 non-null  int64  
 9   Destination_Airport_ID     1592493 non-null  float64
 10  Flight_Cancelled           1631327 non-null  bool   
 11  Departure_State            1631327 non-null  object 
 12  Arrival_State              1631327 non-null  object 
 13  Departure_De

In [70]:
delays = delays[delays['Flight_Cancelled'] == False]
delays.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1581877 entries, 0 to 1631326
Data columns (total 29 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   Weekday                    1581877 non-null  int64  
 1   Month_of_Year              1581877 non-null  int64  
 2   Day_of_Month               1581877 non-null  int64  
 3   Scheduled_Departure_Time   1581877 non-null  int64  
 4   Scheduled_Arrival_Time     1581877 non-null  int64  
 5   Marketing_Airline          1581877 non-null  object 
 6   Marketing_Airline_DOT_ID   1581877 non-null  int64  
 7   Flight_Number              1581877 non-null  int64  
 8   Origin_Airport_ID          1581877 non-null  int64  
 9   Destination_Airport_ID     1544540 non-null  float64
 10  Flight_Cancelled           1581877 non-null  bool   
 11  Departure_State            1581877 non-null  object 
 12  Arrival_State              1581877 non-null  object 
 13  Departure_Delay  

In [71]:
delays = delays[delays['Destination_Airport_ID'].notna()]
delays.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1544540 entries, 0 to 1631326
Data columns (total 29 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   Weekday                    1544540 non-null  int64  
 1   Month_of_Year              1544540 non-null  int64  
 2   Day_of_Month               1544540 non-null  int64  
 3   Scheduled_Departure_Time   1544540 non-null  int64  
 4   Scheduled_Arrival_Time     1544540 non-null  int64  
 5   Marketing_Airline          1544540 non-null  object 
 6   Marketing_Airline_DOT_ID   1544540 non-null  int64  
 7   Flight_Number              1544540 non-null  int64  
 8   Origin_Airport_ID          1544540 non-null  int64  
 9   Destination_Airport_ID     1544540 non-null  float64
 10  Flight_Cancelled           1544540 non-null  bool   
 11  Departure_State            1544540 non-null  object 
 12  Arrival_State              1544540 non-null  object 
 13  Departure_Delay  

In [72]:
delays['Scheduled_Departure_Time'] = delays['Scheduled_Departure_Time'].astype(str).str.zfill(4)
delays['Actual_Departure_Time'] = delays['Actual_Departure_Time'].astype(int).astype(str).str.zfill(4)

delays['sh_h'] = delays['Scheduled_Departure_Time'].str.slice(start=0, stop=2)
delays['sh_min'] = delays['Scheduled_Departure_Time'].str.slice(start=2, stop=4)
delays['loc_h'] = delays['Actual_Departure_Time'].str.slice(start=0, stop=2)
delays['loc_min'] = delays['Actual_Departure_Time'].str.slice(start=2, stop=4)

delays

,Weekday,Month_of_Year,Day_of_Month,Scheduled_Departure_Time,Scheduled_Arrival_Time,Marketing_Airline,Marketing_Airline_DOT_ID,Flight_Number,Origin_Airport_ID,Destination_Airport_ID,Flight_Cancelled,Departure_State,Arrival_State,Departure_Delay,Arrival_Delay,Diverted_Airport_Landings,Taxi_Out_Time,Taxi_In_Time,Flight_Diverted,Actual_Departure_Time,Flight_Duration,Flight_Distance,Origin_Temperature,Destination_Temperature,Origin_Wind_Speed,Destination_Wind_Speed,Origin_Precipitation,Destination_Precipitation,key,sh_h,sh_min,loc_h,loc_min
0,6,6,25,1222,1444,B6,20409,520,10397,12478.0,False,GA,NY,NaN,NaN,0,16.0,10.0,False,1224,107.0,760.0,25.140220,19.357739,21.019808,15.452723,0.0,0.0,B6_520_10397_6_1222_1444,12,22,12,24
1,2,6,21,1216,1304,AA,19805,6297,14107,15376.0,False,AZ,AZ,0.088687,-4.178483,0,16.0,5.0,False,1216,23.0,110.0,13.279939,20.470690,18.045064,12.910265,0.0,0.0,AA_6297_14107_2_1216_1304,12,16,12,16
2,3,1,5,1945,2055,DL,19790,4124,13487,13076.0,False,MN,WI,NaN,5.042185,0,34.0,5.0,False,1945,36.0,119.0,17.816202,13.967273,21.606228,17.976362,0.0,0.1,DL_4124_13487_3_1945_2055,19,45,19,45
3,2,3,22,0700,924,AA,19805,1538,15624,11298.0,False,FL,TX,-1.802698,-0.206932,0,10.0,11.0,False,0658,125.0,641.0,24.562566,14.509228,24.946489,22.630553,0.0,0.0,AA_1538_15624_2_700_924,07,00,06,58
4,4,7,14,2130,2359,AS,19930,1116,14747,12889.0,False,WA,NV,NaN,14.006092,0,23.0,7.0,False,2133,130.0,867.0,8.817992,10.866812,17.426336,17.401007,0.0,0.0,AS_1116_14747_4_2130_2359,21,30,21,33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1631322,7,2,27,0905,1115,AS,19930,1366,14057,12889.0,False,OR,NV,-9.009391,NaN,0,14.0,5.0,False,0856,NaN,763.0,11.703410,11.850033,NaN,NaN,0.1,0.0,AS_1366_14057_7_905_1115,09,05,08,56
1631323,2,4,19,2239,720,AA,19805,276,14771,12478.0,False,CA,NY,-8.042209,-10.015618,0,24.0,30.0,False,2231,285.0,2586.0,33.962022,15.471513,25.784110,24.902274,0.1,0.0,AA_276_14771_2_2239_720,22,39,22,31
1631324,5,6,10,0734,751,UA,19977,5279,11721,13930.0,False,MI,IL,-1.045646,11.909120,0,40.0,9.0,False,0733,41.0,223.0,12.915957,25.351905,11.846814,14.315929,0.0,0.0,UA_5279_11721_5_734_751,07,34,07,33
1631325,3,3,30,1250,1345,AS,19930,421,14747,14057.0,False,WA,OR,-5.099172,-1.992090,0,18.0,11.0,False,1245,29.0,129.0,13.731863,13.932608,16.616156,18.730498,0.0,0.1,AS_421_14747_3_1250_1345,12,50,12,45


In [73]:
delays['Scheduled_Departure_Time'] = delays['Scheduled_Departure_Time'].astype(str).str.zfill(4)
delays['Actual_Departure_Time'] = delays['Actual_Departure_Time'].astype(int).astype(str).str.zfill(4)

delays['sh_h'] = delays['Scheduled_Departure_Time'].str.slice(start=0, stop=2)
delays['sh_min'] = delays['Scheduled_Departure_Time'].str.slice(start=2, stop=4)
delays['loc_h'] = delays['Actual_Departure_Time'].str.slice(start=0, stop=2)
delays['loc_min'] = delays['Actual_Departure_Time'].str.slice(start=2, stop=4)

delays[['sh_h', 'sh_min', 'loc_h', 'loc_min']] = delays[['sh_h', 'sh_min', 'loc_h', 'loc_min']].astype(int)
delays['calculated_delay'] = (delays['loc_h'] - delays['sh_h']) * 60 + (delays['loc_min'] - delays['sh_min'])
delays['Departure_Delay'] = delays['Departure_Delay'].fillna(delays['calculated_delay'])

delays.drop(['sh_h', 'sh_min', 'loc_h', 'loc_min','calculated_delay'], axis =1)

,Weekday,Month_of_Year,Day_of_Month,Scheduled_Departure_Time,Scheduled_Arrival_Time,Marketing_Airline,Marketing_Airline_DOT_ID,Flight_Number,Origin_Airport_ID,Destination_Airport_ID,Flight_Cancelled,Departure_State,Arrival_State,Departure_Delay,Arrival_Delay,Diverted_Airport_Landings,Taxi_Out_Time,Taxi_In_Time,Flight_Diverted,Actual_Departure_Time,Flight_Duration,Flight_Distance,Origin_Temperature,Destination_Temperature,Origin_Wind_Speed,Destination_Wind_Speed,Origin_Precipitation,Destination_Precipitation,key,sh_h,sh_min,loc_h,loc_min,calculated_delay
0,6,6,25,1222,1444,B6,20409,520,10397,12478.0,False,GA,NY,2.000000,NaN,0,16.0,10.0,False,1224,107.0,760.0,25.140220,19.357739,21.019808,15.452723,0.0,0.0,B6_520_10397_6_1222_1444,12,22,12,24,2
1,2,6,21,1216,1304,AA,19805,6297,14107,15376.0,False,AZ,AZ,0.088687,-4.178483,0,16.0,5.0,False,1216,23.0,110.0,13.279939,20.470690,18.045064,12.910265,0.0,0.0,AA_6297_14107_2_1216_1304,12,16,12,16,0
2,3,1,5,1945,2055,DL,19790,4124,13487,13076.0,False,MN,WI,0.000000,5.042185,0,34.0,5.0,False,1945,36.0,119.0,17.816202,13.967273,21.606228,17.976362,0.0,0.1,DL_4124_13487_3_1945_2055,19,45,19,45,0
3,2,3,22,0700,924,AA,19805,1538,15624,11298.0,False,FL,TX,-1.802698,-0.206932,0,10.0,11.0,False,0658,125.0,641.0,24.562566,14.509228,24.946489,22.630553,0.0,0.0,AA_1538_15624_2_700_924,7,0,6,58,-2
4,4,7,14,2130,2359,AS,19930,1116,14747,12889.0,False,WA,NV,3.000000,14.006092,0,23.0,7.0,False,2133,130.0,867.0,8.817992,10.866812,17.426336,17.401007,0.0,0.0,AS_1116_14747_4_2130_2359,21,30,21,33,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1631322,7,2,27,0905,1115,AS,19930,1366,14057,12889.0,False,OR,NV,-9.009391,NaN,0,14.0,5.0,False,0856,NaN,763.0,11.703410,11.850033,NaN,NaN,0.1,0.0,AS_1366_14057_7_905_1115,9,5,8,56,-9
1631323,2,4,19,2239,720,AA,19805,276,14771,12478.0,False,CA,NY,-8.042209,-10.015618,0,24.0,30.0,False,2231,285.0,2586.0,33.962022,15.471513,25.784110,24.902274,0.1,0.0,AA_276_14771_2_2239_720,22,39,22,31,-8
1631324,5,6,10,0734,751,UA,19977,5279,11721,13930.0,False,MI,IL,-1.045646,11.909120,0,40.0,9.0,False,0733,41.0,223.0,12.915957,25.351905,11.846814,14.315929,0.0,0.0,UA_5279_11721_5_734_751,7,34,7,33,-1
1631325,3,3,30,1250,1345,AS,19930,421,14747,14057.0,False,WA,OR,-5.099172,-1.992090,0,18.0,11.0,False,1245,29.0,129.0,13.731863,13.932608,16.616156,18.730498,0.0,0.1,AS_421_14747_3_1250_1345,12,50,12,45,-5


In [74]:
delays.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1544540 entries, 0 to 1631326
Data columns (total 34 columns):
 #   Column                     Non-Null Count    Dtype  
---  ------                     --------------    -----  
 0   Weekday                    1544540 non-null  int64  
 1   Month_of_Year              1544540 non-null  int64  
 2   Day_of_Month               1544540 non-null  int64  
 3   Scheduled_Departure_Time   1544540 non-null  object 
 4   Scheduled_Arrival_Time     1544540 non-null  int64  
 5   Marketing_Airline          1544540 non-null  object 
 6   Marketing_Airline_DOT_ID   1544540 non-null  int64  
 7   Flight_Number              1544540 non-null  int64  
 8   Origin_Airport_ID          1544540 non-null  int64  
 9   Destination_Airport_ID     1544540 non-null  float64
 10  Flight_Cancelled           1544540 non-null  bool   
 11  Departure_State            1544540 non-null  object 
 12  Arrival_State              1544540 non-null  object 
 13  Departure_Delay  

In [75]:
states = pd.read_csv('timechange.csv', index_col=False)
states

,Stan,Strefa
0,NY,7
1,AZ,4
2,WI,7
3,TX,6
4,NV,4
5,CA,4
6,PA,7
7,SD,6
8,AL,6
9,FL,7


In [76]:

delays = pd.merge(delays, states, how='left', left_on='Departure_State', right_on='Stan')
delays.rename(columns={'Strefa': 'StrefaD'}, inplace=True)
delays.drop(columns=['Stan'], inplace=True)

delays = pd.merge(delays, states, how='left', left_on='Arrival_State', right_on='Stan')
delays.rename(columns={'Strefa': 'StrefaA'}, inplace=True)
delays.drop(columns=['Stan'], inplace=True)

delays['Scheduled_Arrival_Time'] = delays['Scheduled_Arrival_Time'].astype(str).str.zfill(4)

delays['arr_h'] = delays['Scheduled_Arrival_Time'].str.slice(start=0, stop=2)
delays['arr_min'] = delays['Scheduled_Arrival_Time'].str.slice(start=2, stop=4)

delays


,Weekday,Month_of_Year,Day_of_Month,Scheduled_Departure_Time,Scheduled_Arrival_Time,Marketing_Airline,Marketing_Airline_DOT_ID,Flight_Number,Origin_Airport_ID,Destination_Airport_ID,Flight_Cancelled,Departure_State,Arrival_State,Departure_Delay,Arrival_Delay,Diverted_Airport_Landings,Taxi_Out_Time,Taxi_In_Time,Flight_Diverted,Actual_Departure_Time,Flight_Duration,Flight_Distance,Origin_Temperature,Destination_Temperature,Origin_Wind_Speed,Destination_Wind_Speed,Origin_Precipitation,Destination_Precipitation,key,sh_h,sh_min,loc_h,loc_min,calculated_delay,StrefaD,StrefaA,arr_h,arr_min
0,6,6,25,1222,1444,B6,20409,520,10397,12478.0,False,GA,NY,2.000000,NaN,0,16.0,10.0,False,1224,107.0,760.0,25.140220,19.357739,21.019808,15.452723,0.0,0.0,B6_520_10397_6_1222_1444,12,22,12,24,2,7,7,14,44
1,2,6,21,1216,1304,AA,19805,6297,14107,15376.0,False,AZ,AZ,0.088687,-4.178483,0,16.0,5.0,False,1216,23.0,110.0,13.279939,20.470690,18.045064,12.910265,0.0,0.0,AA_6297_14107_2_1216_1304,12,16,12,16,0,4,4,13,04
2,3,1,5,1945,2055,DL,19790,4124,13487,13076.0,False,MN,WI,0.000000,5.042185,0,34.0,5.0,False,1945,36.0,119.0,17.816202,13.967273,21.606228,17.976362,0.0,0.1,DL_4124_13487_3_1945_2055,19,45,19,45,0,6,7,20,55
3,2,3,22,0700,0924,AA,19805,1538,15624,11298.0,False,FL,TX,-1.802698,-0.206932,0,10.0,11.0,False,0658,125.0,641.0,24.562566,14.509228,24.946489,22.630553,0.0,0.0,AA_1538_15624_2_700_924,7,0,6,58,-2,7,6,09,24
4,4,7,14,2130,2359,AS,19930,1116,14747,12889.0,False,WA,NV,3.000000,14.006092,0,23.0,7.0,False,2133,130.0,867.0,8.817992,10.866812,17.426336,17.401007,0.0,0.0,AS_1116_14747_4_2130_2359,21,30,21,33,3,4,4,23,59
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1544535,7,2,27,0905,1115,AS,19930,1366,14057,12889.0,False,OR,NV,-9.009391,NaN,0,14.0,5.0,False,0856,NaN,763.0,11.703410,11.850033,NaN,NaN,0.1,0.0,AS_1366_14057_7_905_1115,9,5,8,56,-9,4,4,11,15
1544536,2,4,19,2239,0720,AA,19805,276,14771,12478.0,False,CA,NY,-8.042209,-10.015618,0,24.0,30.0,False,2231,285.0,2586.0,33.962022,15.471513,25.784110,24.902274,0.1,0.0,AA_276_14771_2_2239_720,22,39,22,31,-8,4,7,07,20
1544537,5,6,10,0734,0751,UA,19977,5279,11721,13930.0,False,MI,IL,-1.045646,11.909120,0,40.0,9.0,False,0733,41.0,223.0,12.915957,25.351905,11.846814,14.315929,0.0,0.0,UA_5279_11721_5_734_751,7,34,7,33,-1,7,6,07,51
1544538,3,3,30,1250,1345,AS,19930,421,14747,14057.0,False,WA,OR,-5.099172,-1.992090,0,18.0,11.0,False,1245,29.0,129.0,13.731863,13.932608,16.616156,18.730498,0.0,0.1,AS_421_14747_3_1250_1345,12,50,12,45,-5,4,4,13,45


In [82]:
delays.sort_values(by=['Arrival_Delay'], ascending=False)

,Weekday,Month_of_Year,Day_of_Month,Scheduled_Departure_Time,Scheduled_Arrival_Time,Marketing_Airline,Marketing_Airline_DOT_ID,Flight_Number,Origin_Airport_ID,Destination_Airport_ID,Flight_Cancelled,Departure_State,Arrival_State,Departure_Delay,Arrival_Delay,Diverted_Airport_Landings,Taxi_Out_Time,Taxi_In_Time,Flight_Diverted,Actual_Departure_Time,Flight_Duration,Flight_Distance,Origin_Temperature,Destination_Temperature,Origin_Wind_Speed,Destination_Wind_Speed,Origin_Precipitation,Destination_Precipitation,key,sh_h,sh_min,loc_h,loc_min,calculated_delay,StrefaD,StrefaA,arr_h,arr_min
61930,1,5,23,1004,1158,AA,19805,9677,11278,14321.0,False,VA,ME,7223.210159,7232.230635,0,34.0,6.0,False,1027,83.0,482.0,10.892861,15.739926,17.083283,18.946324,0.1,0.0,AA_9677_11278_1_1004_1158,10,4,10,27,23,7,7,11,58
865282,6,6,18,2105,2254,AA,19805,9631,11298,13277.0,False,TX,AL,-433.000000,5323.698364,0,11.0,9.0,False,1352,86.0,622.0,20.828931,13.442586,20.806240,20.089468,0.0,0.0,AA_9631_11298_6_2105_2254,21,5,13,52,-433,6,6,22,54
897763,6,3,26,0640,1004,AA,19805,2854,11423,11057.0,False,IA,NC,86.000000,2933.899780,0,15.0,6.0,False,0806,91.0,815.0,11.343110,26.110648,22.151979,19.516373,0.0,0.0,AA_2854_11423_6_640_1004,6,40,8,6,86,6,7,10,04
196812,4,6,16,1214,1422,AA,19805,1902,14492,13303.0,False,NC,FL,2618.820772,2677.781618,0,66.0,21.0,False,0753,100.0,700.0,15.427082,14.251073,17.481301,26.679210,0.1,0.0,AA_1902_14492_4_1214_1422,12,14,7,53,-261,7,7,14,22
1429419,1,7,11,1145,1341,AA,19805,2730,15096,11057.0,False,NY,NC,2555.933698,2562.223524,0,20.0,7.0,False,0621,95.0,603.0,21.996580,20.203779,21.682623,24.101675,0.0,0.1,AA_2730_15096_1_1145_1341,11,45,6,21,-324,7,7,13,41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1544520,3,5,4,1154,1320,UA,19977,2326,15304,12266.0,False,FL,TX,-4.876249,NaN,0,9.0,5.0,False,1149,120.0,787.0,27.051648,13.764391,19.369911,24.049843,0.0,0.0,UA_2326_15304_3_1154_1320,11,54,11,49,-5,7,6,13,20
1544521,7,1,2,0855,1213,B6,20409,201,12478,11697.0,False,NY,FL,0.003874,NaN,0,39.0,10.0,False,0855,160.0,1069.0,7.527444,14.769264,20.914892,28.442786,0.0,0.1,B6_201_12478_7_855_1213,8,55,8,55,0,7,7,12,13
1544527,2,4,12,1115,1214,F9,20436,1303,13204,14635.0,False,FL,FL,-7.000000,NaN,0,12.0,7.0,False,1108,31.0,133.0,15.074849,13.949699,14.011743,14.667089,0.0,0.0,F9_1303_13204_2_1115_1214,11,15,11,8,-7,7,7,12,14
1544528,1,4,4,1640,1923,UA,19977,2095,10423,13930.0,False,TX,IL,3.444472,NaN,0,13.0,10.0,False,1644,131.0,977.0,18.330330,14.661134,19.887094,18.403446,0.5,0.0,UA_2095_10423_1_1640_1923,16,40,16,44,4,6,6,19,23


In [ ]:
delays2 = delays.sort()

wylatujemy o x godzinie 